In [ ]:
# Q1 final version
# (1) existing 0–5 = infant + toddler + preschool
# (2) expansion upper bound: eub_f = min(1.2*n_f, 500)
# (3) delete dummy and add expansion_term  --> NOW PIECEWISE

import pandas as pd
from gurobipy import Model, GRB, quicksum

# Data reading and cleaning
pop = pd.read_csv('population.csv')
pop['zipcode'] = pop['zipcode'].astype(str).str[:5].str.zfill(5)
pop['pop_0_5'] = pop['-5']
pop['pop_0_12'] = pop['-5'] + pop['5-9'] + 0.6 * pop['10-14']
pop = pop[['zipcode', 'pop_0_5', 'pop_0_12']]

inc = pd.read_csv('avg_individual_income.csv')
inc.rename(columns={'ZIP code': 'zipcode', 'average income': 'avg_income'}, inplace=True)
inc['zipcode'] = inc['zipcode'].astype(str).str[:5].str.zfill(5)

# use average instead of NaN
inc['avg_income'] = pd.to_numeric(inc['avg_income'], errors='coerce')
mean_income = inc['avg_income'].mean(skipna=True)
inc['avg_income'] = inc['avg_income'].fillna(mean_income)

emp = pd.read_csv('employment_rate.csv')
emp.rename(columns={'employment rate': 'employment_rate'}, inplace=True)
emp['zipcode'] = emp['zipcode'].astype(str).str[:5].str.zfill(5)

# if rate is greater than 1, divide by 100; use average instead of NaN
emp['employment_rate'] = pd.to_numeric(emp['employment_rate'], errors='coerce')
if emp['employment_rate'].max(skipna=True) > 1.0:
    emp['employment_rate'] = emp['employment_rate'] / 100.0
mean_rate = emp['employment_rate'].mean(skipna=True)
emp['employment_rate'] = emp['employment_rate'].fillna(mean_rate).clip(0, 1)

fac = pd.read_csv('child_care_regulated.csv')
fac.rename(columns={'zip_code': 'zipcode'}, inplace=True)
fac['zipcode'] = fac['zipcode'].astype(str).str[:5].str.zfill(5)
fac['capacity'] = fac['total_capacity']

for col in ['infant_capacity', 'toddler_capacity', 'preschool_capacity']:
    if col not in fac.columns:
        fac[col] = 0.0
    fac[col] = fac[col].fillna(0.0)

# existing 0–5 at facility level
fac['exist_05'] = (fac['infant_capacity'] +fac['toddler_capacity'] +fac['preschool_capacity'])
fac = fac[['facility_id', 'program_type', 'zipcode','capacity', 'latitude', 'longitude', 'exist_05']]

pot = pd.read_csv('potential_locations.csv')
pot['zipcode'] = pot['zipcode'].astype(str).str[:5].str.zfill(5)
pot = pot[['zipcode', 'latitude', 'longitude']].reset_index(drop=False).rename(columns={'index': 'loc_id'})

# Merge
z_df = (pop.merge(inc, on='zipcode', how='left')
       .merge(emp, on='zipcode', how='left'))

z_df['theta'] = ((z_df['employment_rate'] >= 0.6) |(z_df['avg_income'] <= 60000)).map({True: 0.5, False: 1/3})



# Sets and Parameters
Z = list(z_df['zipcode'])
F = list(fac['facility_id'])
L = list(pot['loc_id'])
S = ['S','M','L']

Pop05  = dict(zip(z_df['zipcode'], z_df['pop_0_5'].astype(float)))
Pop012 = dict(zip(z_df['zipcode'], z_df['pop_0_12'].astype(float)))
theta  = dict(zip(z_df['zipcode'], z_df['theta'].astype(float)))

z_of_f = dict(zip(fac['facility_id'], fac['zipcode']))
n_f    = dict(zip(fac['facility_id'], fac['capacity'].astype(float)))

eub = {f: min(1.2 * n_f[f], 500.0) for f in F}  # upper bound on e

z_of_l = dict(zip(pot['loc_id'], pot['zipcode']))

Cap_tot = {'S':100.0, 'M':200.0, 'L':400.0}
Cap_05  = {'S':50.0,  'M':100.0, 'L':200.0}
C_build = {'S':65000.0, 'M':95000.0, 'L':115000.0}

alpha = 2.0/3.0
C_expand_fixed = 20000.0
C_expand_scale_coeff = 200.0
C_add_05 = 100.0

F_by_z = {z: [f for f in F if z_of_f[f]==z] for z in Z}
L_by_z = {z: [l for l in L if z_of_l[l]==z] for z in Z}

exist05_by_z    = fac.groupby('zipcode')['exist_05'].sum().to_dict()
existTotal_by_z = fac.groupby('zipcode')['capacity'].sum().to_dict()
for z in Z:
    exist05_by_z.setdefault(z, 0.0)
    existTotal_by_z.setdefault(z, 0.0)



# Variables
m = Model('Q1_final_piecewise')
m.Params.OutputFlag = 1

# New facilities
y = m.addVars(L, S, vtype=GRB.BINARY, name='y')
x_new_05  = m.addVars(L, S, lb=0.0, vtype=GRB.CONTINUOUS, name='x_new_05')
x_new_512 = m.addVars(L, S, lb=0.0, vtype=GRB.CONTINUOUS, name='x_new_512')

# Existing facilities (expansion)
e = m.addVars(F, lb=0.0, vtype=GRB.CONTINUOUS, name='e')        # added slots
x_ext_05  = m.addVars(F, lb=0.0, vtype=GRB.CONTINUOUS, name='x_ext_05')
x_ext_512 = m.addVars(F, lb=0.0, vtype=GRB.CONTINUOUS, name='x_ext_512')



# Objective Function (build + piecewise expansion + 0-5 add cost)
build_cost = quicksum(C_build[s]*y[l,s] for l in L for s in S)

# ----- PIECEWISE EXPANSION COST (delta1/delta2) -----
# 1) define auxiliaries
delta1 = m.addVars(F, lb=0.0, vtype=GRB.CONTINUOUS, name='delta1')  # first regime: up to min(n_f, eub)
delta2 = m.addVars(F, lb=0.0, vtype=GRB.CONTINUOUS, name='delta2')  # second regime: remaining expansion

# 2) link and caps
for f in F:
    # split expansion across two regimes
    m.addConstr(x_ext_05[f] + x_ext_512[f] == delta1[f] + delta2[f], name=f'piece_link[{f}]')
    # regime-1 cap: at most min(n_f, eub[f])
    m.addConstr(delta1[f] <= min(n_f[f], eub[f]), name=f'piece_cap1[{f}]')
    # regime-2 cap: remaining up to eub[f]
    m.addConstr(delta2[f] <= max(eub[f] - min(n_f[f], eub[f]), 0.0), name=f'piece_cap2[{f}]')

# 3) expansion cost using two slopes (replace old proportional term)
expansion_term = quicksum(
    (((C_expand_fixed / n_f[f]) + C_expand_scale_coeff) * delta1[f] + C_expand_scale_coeff * delta2[f])
    for f in F if n_f[f] > 0.0
)

# 0–5 equipment cost
add05_cost = C_add_05 * (
    quicksum(x_new_05[l,s] for l in L for s in S) +
    quicksum(x_ext_05[f]   for f in F)
)

m.setObjective(build_cost + expansion_term + add05_cost, GRB.MINIMIZE)



# Constraints
for l in L:
    m.addConstr(quicksum(y[l,s] for s in S) <= 1, name=f'one_size[{l}]')

# 1) new facility capacities
for l in L:
    for s in S:
        m.addConstr(x_new_05[l,s] + x_new_512[l,s] <= Cap_tot[s]*y[l,s], name=f'new_tot[{l},{s}]')
        m.addConstr(x_new_05[l,s] <= Cap_05[s]*y[l,s], name=f'new_05_cap[{l},{s}]')

# 2) expansion linkage & caps
for f in F:
    m.addConstr(x_ext_05[f] + x_ext_512[f] == e[f], name=f'ext_link[{f}]')
    m.addConstr(e[f] <= eub[f], name=f'ext_cap[{f}]')

# 3) eliminate deserts (0–12)
for z in Z:
    existing = existTotal_by_z[z]
    exp_add  = quicksum(e[f] for f in F_by_z[z])
    new_add  = quicksum(x_new_05[l,s] + x_new_512[l,s] for l in L_by_z[z] for s in S)
    m.addConstr(existing + exp_add + new_add >= theta[z]*Pop012[z], name=f'desert[{z}]')

# 4) 0–5 coverage
for z in Z:
    add05 = quicksum(x_ext_05[f] for f in F_by_z[z]) \
          + quicksum(x_new_05[l,s] for l in L_by_z[z] for s in S)
    m.addConstr(exist05_by_z[z] + add05 >= alpha*Pop05[z], name=f'cov05[{z}]')

# Solve
m.optimize()

# Report
if m.status == GRB.OPTIMAL:
    total_cost = m.objVal

    cost_new   = sum(C_build[s]*y[l,s].X for l in L for s in S)
    # piecewise expansion cost consistent with objective
    cost_exp   = sum(
        (((C_expand_fixed / n_f[f]) + C_expand_scale_coeff) * delta1[f].X + C_expand_scale_coeff * delta2[f].X)
        for f in F if n_f[f] > 0.0
    )
    cost_add05 = 100.0 * (sum(x_new_05[l,s].X for l in L for s in S)
                          + sum(x_ext_05[f].X   for f in F))

    expanded = [f for f in F if (x_ext_05[f].X + x_ext_512[f].X) > 1e-6]
    avg_e    = (sum(e[f].X for f in expanded) / len(expanded)) if expanded else 0.0
    avg_ratio= (sum((x_ext_05[f].X + x_ext_512[f].X)/n_f[f] for f in expanded) / len(expanded)) if expanded else 0.0
    
    print(f"\nOptimal Total Cost:      ${total_cost:,.2f}")
    print(f"  - Construction:          ${cost_new:,.2f}")
    print(f"  - Expansion (piecewise):  ${cost_exp:,.2f}")
    print(f"  - 0–5 equipment:         ${cost_add05:,.2f}")
    print(f"  - Expanded facilities:   {len(expanded)}; "
          f"avg added = {avg_e:.1f} slots; avg expansion ratio = {avg_ratio:.3f}")


    chosen_new = [(l, s) for l in L for s in S if y[l, s].X > 0.5]
    print(f"\nNew sites (first 15): {chosen_new[:15]}{' ...' if len(chosen_new) > 15 else ''}")


# (Optional) write model to LP for inspection
m.write("Q1_piecewise.lp")
print("LP written to Q1_piecewise.lp")


Set parameter Username
Set parameter LicenseID to value 2709126
Academic license - for non-commercial use only - expires 2026-09-15
Set parameter OutputFlag to value 1
